# Use Machine Learning in a Trading Strategy

## Part 1
### Loading and Preparing the Data

In [1]:
# Import required libraries
import pandas as pd
import time
import datetime
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import mplfinance as mpf
import hvplot.pandas
import math

import yfinance as yf

C:\ProgramData\Anaconda3\envs\dev\lib\site-packages\pyarrow\compute.py:198: RuntimeWarning: Python binding for CumulativeSumOptions not exposed
  .format(class_name), RuntimeWarning)
C:\ProgramData\Anaconda3\envs\dev\lib\site-packages\pyarrow\compute.py:198: RuntimeWarning: Python binding for ListSliceOptions not exposed
  .format(class_name), RuntimeWarning)
C:\ProgramData\Anaconda3\envs\dev\lib\site-packages\pyarrow\compute.py:198: RuntimeWarning: Python binding for RankOptions not exposed
  .format(class_name), RuntimeWarning)

Bad key text.latex.preview in file C:\ProgramData\Anaconda3\envs\dev\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file C:\ProgramData\Anaconda3\envs\dev\lib\site-packages\matplotlib\mp

ModuleNotFoundError: No module named 'mplfinance'

In [ ]:
# Import the dataset in 5 minute intervals into a Pandas Dataframe
ticker = yf.Ticker("BTC-USD")
BTC_df = ticker.history(period='max', interval='5m', start='2022-12-01', end='2023-01-24')

# Display sample data
BTC_df.head()

In [ ]:
colors = mpf.make_marketcolors(up='green',
                               down = 'red',
                               wick = 'inherit',
                               edge='inherit',
                               volume = 'in')
mpf_style = mpf.make_mpf_style(base_mpf_style = 'yahoo', marketcolors = colors)
mpf.plot(BTC_df['2023-01-23':], type = 'candle', style = mpf_style, volume=True)

In [ ]:
# Drop unnecessary columns and rename the remaining columns
BTC_df = BTC_df.drop(columns =["Open", "High", "Low", "Dividends", "Stock Splits"], axis=1)
BTC_df.head()

In [ ]:
# Calculate the 5 min returns
BTC_df["actual_returns"] = BTC_df["Close"].pct_change()
BTC_df = BTC_df.dropna()
BTC_df.head()

### Generating the Features and Target Sets

#### Creating the Features Set

In [ ]:
# Define a window sizes
short_window = 20
long_window = 50

# Create SMAs that use short_window, long_window, and assign to new columns 
BTC_df["sma_fast"] = BTC_df["Close"].rolling(window=short_window).mean()
BTC_df["sma_slow"] = BTC_df["Close"].rolling(window=long_window).mean()

# Drop the NaNs using dropna()
BTC_df = BTC_df.dropna()
BTC_df

In [ ]:
# Assign a copy of the `sma_fast` and `sma_slow` columns to a new DataFrame called `X`
X = BTC_df[["Close","sma_fast", "sma_slow"]].shift().dropna().copy()

# Display sample data
display(X.head())
display(X.tail())

In [ ]:
# Visualize close price for the investment
BTC_close = X[["Close"]].hvplot(
    line_color='black',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = X[["sma_fast", "sma_slow"]].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
sma_ema_plot = BTC_close * moving_avgs
sma_ema_plot

In [ ]:
import hvplot.pandas 
X = X[["sma_fast", "sma_slow"]]
X.hvplot()

#### Creating the Target Set

In [ ]:
# Create a new column in the `trading_df` called "signal" setting its value to zero.
BTC_df["Signal"] = 0.0

In [ ]:
# Create the signal to buy and sell
BTC_df.loc[((BTC_df["sma_fast"] > BTC_df["sma_slow"]) & (BTC_df["Close"] > BTC_df["sma_fast"])), "Signal"] = 1
BTC_df.loc[((BTC_df["sma_fast"] > BTC_df["sma_slow"]) & (BTC_df["Close"] < BTC_df["sma_fast"])), "Signal"] = -1
BTC_df.loc[((BTC_df["sma_fast"] < BTC_df["sma_slow"]) & (BTC_df["Close"] < BTC_df["sma_fast"])), "Signal"] = -1
BTC_df.loc[((BTC_df["sma_fast"] < BTC_df["sma_slow"]) & (BTC_df["Close"] > BTC_df["sma_fast"])), "Signal"] = 1


#ticker_df.loc[(ticker_df["sma_fast"] > ticker_df["sma_slow"]), "Signal"] = 1
#ticker_df.loc[(ticker_df["sma_fast"] < ticker_df["sma_slow"]), "Signal"] = -1
df = pd.DataFrame(BTC_df["Signal"])
df

In [ ]:
# Copy the new "signal" column to a new Series called `y`.
y = BTC_df["Signal"].copy()
y.value_counts()

### Split the Data into Training and Testing Sets

In [ ]:
# Import required libraries
from pandas.tseries.offsets import DateOffset

In [ ]:
# Select the start of the training period
training_begin = X.index.min()

# Display the training begin date
print(training_begin)

In [ ]:
# Select the ending period for the training data with an offset of 25 Days
training_end = X.index.min() + DateOffset(days=25)

# Display the training end date
print(training_end)

In [ ]:
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

In [ ]:
# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end:]
y_test = y.loc[training_end:]

### Standardize the Data

In [ ]:
# Import required libraries
from sklearn.preprocessing import StandardScaler

In [ ]:
# Create a StandardScaler instance
scaler = StandardScaler()
 
# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)
 
# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
X_train

---

## Part 2
### Using Machine Learning Into a Trading Strategy

In [ ]:
# Import the SVM model
from sklearn import svm
from sklearn.metrics import classification_report

In [ ]:
# Create the classifier model
svm_model = svm.SVC()

# Fit the model to the data using X_train_scaled and y_train
svm_model = svm_model.fit(X_train_scaled, y_train)

# Use the trained model to predict the trading signals for the training data
training_signal_predictions = svm_model.predict(X_train_scaled)

# Display the sample predictions
training_signal_predictions[:10]

In [ ]:
# Evaluate the model using a classification report
training_report = classification_report(y_train, training_signal_predictions)

# Display report
print(training_report)

#### Backtesting a Machine Learning Trading Algorithm

In [ ]:
# Use the trained model to predict the trading signals for the testing data.
testing_signal_predictions = svm_model.predict(X_test_scaled)

In [ ]:
# Evaluate the model's ability to predict the trading signal for the testing data
testing_report = classification_report(y_test, testing_signal_predictions)

# Display the report
print(testing_report)

#### Visually Compare the Actual and Predicted Returns

In [ ]:
# Create a predictions DataFrame
predictions_df = pd.DataFrame(index=X_test.index)

predictions_df["predicted_signal"] = testing_signal_predictions

predictions_df["actual_returns"] = BTC_df["actual_returns"]

predictions_df["trading_algorithm_returns"] = (
    predictions_df["actual_returns"] * predictions_df["predicted_signal"]
)

# Review the DataFrame
predictions_df

In [ ]:
# Calculate and plot the cumulative returns for the `actual_returns` and the `trading_algorithm_returns`
(1 + predictions_df[["actual_returns", "trading_algorithm_returns"]]).cumprod().hvplot()

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation

In [ ]:
# Build the neural network layers
model = Sequential()

# Define the the number of inputs to the model
number_inputs = 2

# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 = 64

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = 32

# Create the Sequential model instance
model_seq = Sequential()

# Add the first Dense layer specifying the number of inputs, the number of hidden nodes, and the activation function
model_seq.add(Dense(units=hidden_nodes_layer1, input_dim=number_inputs, activation="relu"))

# Add the second Dense layer specifying the number of hidden nodes and the activation function
model_seq.add(Dense(units=hidden_nodes_layer2, activation="relu"))

# Add the second Dense layer specifying the number of hidden nodes and the activation function
model_seq.add(Dense(units=24, activation="relu"))

# Add the second Dense layer specifying the number of hidden nodes and the activation function
model_seq.add(Dense(units=12, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
model_seq.add(Dense(1))

In [ ]:
# Summarise the structure of the model
model_seq.summary()

In [ ]:
# Compile the Sequential model
nn.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Fit the model
nn.fit(X_train_scaled,y_train, 
                    epochs=20,
                    batch_size=100,
                    shuffle=True)